In [1]:
import re
import requests
import os
import pandas as pd
import pyarrow.parquet as pq

In [2]:
txt = r"""
<pre><a href="../">../</a>
<a href="part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00001-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00001-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00002-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00002-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00003-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00003-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00004-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00004-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00005-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00005-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00006-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00006-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00007-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00007-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00008-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00008-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00009-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00009-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00010-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00010-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00011-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00011-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00012-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00012-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00013-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00013-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00014-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00014-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00015-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00015-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00016-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00016-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00017-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00017-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00018-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00018-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00019-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00019-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00020-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00020-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00021-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00021-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00022-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00022-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00023-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00023-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00024-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00024-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00025-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00025-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00026-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00026-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00027-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00027-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00028-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00028-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00029-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00029-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00030-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00030-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
<a href="part-00031-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet">part-00031-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36..&gt;</a> 01-Jan-1970 00:00      2G
</pre>
"""

def download_file(url: str, destination_path: str):
    if os.path.exists(destination_path):
        print("Exists, skipping: " + url)
        return
    print("Downloading: " + url)
    response = requests.get(url)
    print("Saving to: " + destination_path)
    with open(destination_path, "wb") as f:
        f.write(response.content)
    print("OK\n")

domain = "https://the-eye.eu/public/AI/cah/laion400m-met-release/laion400m-meta/"
save_path = r"D:\laion400m_parquests"
pattern = r"part-\d+\-.+.parquet"
res = re.findall(pattern, txt)
for file_name in res:
    url = domain + file_name
    destination_path = os.path.join(save_path, file_name)
    download_file(url, destination_path)


Exists, skipping: https://the-eye.eu/public/AI/cah/laion400m-met-release/laion400m-meta/part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
Exists, skipping: https://the-eye.eu/public/AI/cah/laion400m-met-release/laion400m-meta/part-00001-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
Exists, skipping: https://the-eye.eu/public/AI/cah/laion400m-met-release/laion400m-meta/part-00002-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
Exists, skipping: https://the-eye.eu/public/AI/cah/laion400m-met-release/laion400m-meta/part-00003-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
Exists, skipping: https://the-eye.eu/public/AI/cah/laion400m-met-release/laion400m-meta/part-00004-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
Exists, skipping: https://the-eye.eu/public/AI/cah/laion400m-met-release/laion400m-meta/part-00005-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
Exists, skipping: https://the-eye.eu/public/AI/cah/laion400m-met

In [ ]:
fname = "part-00001-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet"
path = r"D:\laion400m_parquests"
parquet_path = os.path.join(path, fname)

table = pq.read_table(parquet_path)

SAMPLE_ID: int64
URL: string
TEXT: string
HEIGHT: int32
WIDTH: int32
LICENSE: string
NSFW: string
similarity: double
-- schema metadata --
org.apache.spark.version: '3.1.2'
org.apache.spark.sql.parquet.row.metadata: '{"type":"struct","fields":[{"' + 511
             SAMPLE_ID                                                URL  \
0         3.754690e+12  https://photos6.spartoo.co.uk/photos/218/21855...   
1         2.438638e+12  https://upload.wikimedia.org/wikipedia/commons...   
2         3.325701e+12    http://i.ytimg.com/vi/PcboCDaJlu0/mqdefault.jpg   
3         2.021529e+12  http://ladyenslain.net/festivals/2011/07.22-24...   
4         4.264074e+12  https://stp.sgp1.digitaloceanspaces.com/storag...   
...                ...                                                ...   
12933514  2.263458e+12  https://cdn.images.express.co.uk/img/dynamic/1...   
12933515  4.048350e+12  https://i.pinimg.com/236x/84/a5/1e/84a51e6114a...   
12933516  4.207729e+12  https://www.originalsoundvers

In [ ]:
print(table.schema)     # view columns
print(table.to_pandas())  # convert to pandas if needed
print(table.column_names)